<h1>Data Preprocessing<h1>

In [1]:
#import libraries
import pandas as pd
import json
from sqlalchemy import create_engine, text 
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessor as p
from sklearn.model_selection import GridSearchCV
import gensim
import re
from gensim.parsing.preprocessing import remove_stopwords
import string

In [2]:
#create connection object to postgres server
ip='127.0.0.1' 
engine = create_engine('postgresql://postgres:anupam@'+ip+':5432/twitter')
connection = engine.connect()

In [3]:
#helper to fetch postgres data
def get_data_from_postgres(data_query):
    result = connection.execute(data_query).fetchall()
    df = pd.DataFrame(result)
    return df

In [7]:
#helper to clean tweets
def tweet_clean(row):
    tweet = row['tweet_original'].lower()
    tweet = p.clean(tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = remove_stopwords(tweet)
    return tweet

In [10]:
#get data from postgres
query = text("select * from tweets_full")
data = get_data_from_postgres(query)

In [11]:
#rename columns
data = data.rename(columns={'manual_relevant':'related_manual','related':'related_score_based','msg':'tweet_original'})
data

,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,tweet_original,context,sensitive,created_at,ccs_score,rowid,relevant_manualscore,related_score_based,related_manual
0,152,948621816830005248,62a2a7f86cd9a5b4,1641001927528808450,en,15,0,1,0,@Lekjutter @GrassWarehouse @ASA_UK 1. There ar...,None,False,2023-03-29 08:58:42,0,798,None,0,1.0
1,196,1150771325721546754,7d7bdec12d2549d4,1640409818140639232,en,6,0,0,1,"In Feb, ground was broken on the new Bluecoat ...",None,False,2023-03-27 17:45:52,0,1019,None,0,1.0
2,50401,561869540,022566532a1dff30,1567144971597144067,en,0,0,0,0,Hear Hear.. Total letdown!! https://t.co/ZMPzF...,None,False,2022-09-06 13:37:32,0,251268,None,0,NaN
3,50401,1397888324,00486f39ae8bd30d,542818661787254784,en,0,0,0,0,"Ooh I have a new follower, never heard of them...",None,False,2014-12-10 23:10:28,0,251269,None,0,NaN
4,63285,3374556155,06168d1feda43857,1549099736573042688,en,3,0,0,0,@mrsbaileywrites @CressMcLaughlin Mine is a to...,None,False,2022-07-18 18:32:12,0,315592,None,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267784,31367,765488234,71c8eb57c400c9b6,1331880999182069760,en,1,0,0,0,A great sportsman and real man of steel! Shoul...,"[{""domain"": {""id"": ""11"", ""name"": ""Sport"", ""des...",False,2020-11-26 08:42:30,0,156249,None,0,NaN
1267785,31367,1180399555,604492c5e49c512b,1588448050346831872,en,3,0,0,1,@Ottojizzmark I don't think Matt Handcock know...,None,False,2022-11-04 08:28:21,0,156250,None,0,NaN
1267786,31367,2653892275,7ae9e2f2ff7a87cd,909897562475388929,en,1,0,0,0,Glad to hear you had a good time @MalcParry1! ...,None,False,2017-09-18 21:50:58,0,156251,None,0,NaN
1267787,31368,18139271,3bc1b6cfd27ef7f6,1588447975722082304,en,1,0,1,0,@wyadvd @1980sRewind I think BP was second las...,None,False,2022-11-04 08:28:04,0,156252,None,0,NaN


In [13]:
data['related_manual'].value_counts()

related_manual
1.0    1250
0.0    1250
Name: count, dtype: int64

In [14]:
#clean the tweets
data['tweet_preprocessed'] = data.apply(tweet_clean, axis=1)

In [28]:
#tokenize raw tweets
data['tweet_tokenized_original']=[word_tokenize(entry) for entry in data['tweet_original']]

In [19]:
#tokenize raw tweets
data['tweet_tokenized_preprocessed']=[word_tokenize(entry) for entry in data['tweet_preprocessed']]

In [20]:
#helper to generate word vector
def get_word_vector(tokens, size, model):
    vector = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vector += model[word].reshape((1, size)) #add the word2vec vector for each single word in the tweet
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vector /= count #take the mean to get the word2vec representation of a tweet
    return vector

In [32]:
#train word2vec model on the tokenized data
%%time
word2vector_model_original = gensim.models.Word2Vec(
            data['tweet_tokenized_original'],
            size=300,
            window=5,
            min_count=2,                                  
            sg = 1,
            hs = 0,
            negative = 10,
            workers= 5,
            seed = 34
) 
word2vector_model_original.train(data['tweet_tokenized_original'], total_examples= len(data['tweet_tokenized_original']), epochs=20)

CPU times: user 4h 41min 25s, sys: 59.1 s, total: 4h 42min 24s
Wall time: 57min 54s


(499917520, 685366500)

In [33]:
%%time
vectors_original = np.zeros((len(data['tweet_tokenized_original']), 300)) 
for i in range(len(data['tweet_tokenized_original'])):
    vectors_original[i,:] = get_word_vector(data['tweet_tokenized_original'][i], 300, word2vector_model_original)

/var/folders/yp/w3wbm1755g3dkb_6mbfzm41r0000gn/T/ipykernel_85207/4174304635.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += model[word].reshape((1, size))


CPU times: user 3min 7s, sys: 2.78 s, total: 3min 10s
Wall time: 3min 11s


In [34]:
arr = {"lists": list(vectors_original)}
tweet_word2vec_original = pd.DataFrame(arr, columns=['lists'])
data['tweet_word2vec_original'] = tweet_word2vec_original['lists']
data

,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,tweet_original,...,created_at,ccs_score,rowid,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_original,tweet_tokenized_preprocessed,tweet_word2vec_original
0,152,948621816830005248,62a2a7f86cd9a5b4,1641001927528808450,en,15,0,1,0,@Lekjutter @GrassWarehouse @ASA_UK 1. There ar...,...,2023-03-29 08:58:42,0,798,None,0,1.0,lekjutter grasswarehouse asauk nonelectricfuel...,"[@, Lekjutter, @, GrassWarehouse, @, ASA_UK, 1...","[lekjutter, grasswarehouse, asauk, nonelectric...","[0.0401556013490666, -0.013895715402693234, 0...."
1,196,1150771325721546754,7d7bdec12d2549d4,1640409818140639232,en,6,0,0,1,"In Feb, ground was broken on the new Bluecoat ...",...,2023-03-27 17:45:52,0,1019,None,0,1.0,feb ground broken new bluecoat academy carring...,"[In, Feb, ,, ground, was, broken, on, the, new...","[feb, ground, broken, new, bluecoat, academy, ...","[0.06797821347468666, 0.07370191409375236, 0.1..."
2,50401,561869540,022566532a1dff30,1567144971597144067,en,0,0,0,0,Hear Hear.. Total letdown!! https://t.co/ZMPzF...,...,2022-09-06 13:37:32,0,251268,None,0,NaN,hear hear total letdown httpstcozmpzfydkr6,"[Hear, Hear, .., Total, letdown, !, !, https, ...","[hear, hear, total, letdown, httpstcozmpzfydkr6]","[0.1666468804081281, -0.062414234711064234, 0...."
3,50401,1397888324,00486f39ae8bd30d,542818661787254784,en,0,0,0,0,"Ooh I have a new follower, never heard of them...",...,2014-12-10 23:10:28,0,251269,None,0,NaN,ooh new follower heard bound nutter takes tota...,"[Ooh, I, have, a, new, follower, ,, never, hea...","[ooh, new, follower, heard, bound, nutter, tak...","[0.02465900614236792, -0.006915513542480767, 0..."
4,63285,3374556155,06168d1feda43857,1549099736573042688,en,3,0,0,0,@mrsbaileywrites @CressMcLaughlin Mine is a to...,...,2022-07-18 18:32:12,0,315592,None,0,NaN,mrsbaileywrites cressmclaughlin total star wou...,"[@, mrsbaileywrites, @, CressMcLaughlin, Mine,...","[mrsbaileywrites, cressmclaughlin, total, star...","[0.012976075936522748, -0.04400134908953785, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267784,31367,765488234,71c8eb57c400c9b6,1331880999182069760,en,1,0,0,0,A great sportsman and real man of steel! Shoul...,...,2020-11-26 08:42:30,0,156249,None,0,NaN,great sportsman real man steel nominated https...,"[A, great, sportsman, and, real, man, of, stee...","[great, sportsman, real, man, steel, nominated...","[-0.04416359712680181, 0.011064327182248234, -..."
1267785,31367,1180399555,604492c5e49c512b,1588448050346831872,en,3,0,0,1,@Ottojizzmark I don't think Matt Handcock know...,...,2022-11-04 08:28:21,0,156250,None,0,NaN,ottojizzmark dont think matt handcock knows lo...,"[@, Ottojizzmark, I, do, n't, think, Matt, Han...","[ottojizzmark, dont, think, matt, handcock, kn...","[0.08948575894526661, 0.008041062379909798, 0...."
1267786,31367,2653892275,7ae9e2f2ff7a87cd,909897562475388929,en,1,0,0,0,Glad to hear you had a good time @MalcParry1! ...,...,2017-09-18 21:50:58,0,156251,None,0,NaN,glad hear good time malcparry1 total blast bri...,"[Glad, to, hear, you, had, a, good, time, @, M...","[glad, hear, good, time, malcparry1, total, bl...","[0.02253250631949176, -0.05413341651792112, 0...."
1267787,31368,18139271,3bc1b6cfd27ef7f6,1588447975722082304,en,1,0,1,0,@wyadvd @1980sRewind I think BP was second las...,...,2022-11-04 08:28:04,0,156252,None,0,NaN,wyadvd srewind think bp second neighbours,"[@, wyadvd, @, 1980sRewind, I, think, BP, was,...","[wyadvd, srewind, think, bp, second, neighbours]","[-0.01564978822492636, -0.034952578922876947, ..."


In [21]:
#repeat the process for cleaned tweets
%%time
word2vector_model_preprocessed = gensim.models.Word2Vec(
            data['tweet_tokenized_preprocessed'],
            size=300,
            window=5,
            min_count=2,                                  
            sg = 1,
            hs = 0,
            negative = 10,
            workers= 32,
            seed = 34
) 

word2vector_model_preprocessed.train(data['tweet_tokenized_preprocessed'], total_examples= len(data['tweet_tokenized_preprocessed']), epochs=20)

CPU times: user 2h 40min 14s, sys: 58.2 s, total: 2h 41min 13s
Wall time: 15min 16s


(227041004, 250550140)

In [22]:
%%time
vectors_preprocessed = np.zeros((len(data['tweet_tokenized_preprocessed']), 300)) 
for i in range(len(data['tweet_tokenized_preprocessed'])):
    vectors_preprocessed[i,:] = get_word_vector(data['tweet_tokenized_preprocessed'][i], 300, word2vector_model_preprocessed)

/var/folders/yp/w3wbm1755g3dkb_6mbfzm41r0000gn/T/ipykernel_34846/4174304635.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += model[word].reshape((1, size))


CPU times: user 1min 29s, sys: 1.9 s, total: 1min 31s
Wall time: 1min 31s


In [23]:
arr = {"lists": list(vectors_preprocessed)}
tweet_word2vec_preprocessed = pd.DataFrame(arr, columns=['lists'])
data['tweet_word2vec_preprocessed'] = tweet_word2vec_preprocessed['lists']

In [17]:
data

,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,tweet_original,...,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_original,tweet_tokenized_preprocessed,tweet_word2vec_original,tweet_word2vec_preprocessed,context_domain_name,context_entity_name
0,31371,58555376,1a77876636535465,574226197921595394,en,0,0,0,1,Pen 26 Garn 3\n\nBp in the bag\n\n#pitmen\n\n#fb,...,None,0,NaN,pen garn bp bagpitmenfb,"[Pen, 26, Garn, 3, Bp, in, the, bag, #, pitmen...","[pen, garn, bp, bagpitmenfb]","[0.09151028875600208, 0.04513007774949074, -0....","[0.004402711604294988, -0.012395232915878296, ...",[unknown],[unknown]
1,31376,25727727,1da00c8852cc9da2,574217464243949568,en,2,0,1,0,Definitely going to look like a total wally on...,...,None,0,NaN,definitely going look like total wally train c...,"[Definitely, going, to, look, like, a, total, ...","[definitely, going, look, like, total, wally, ...","[-0.03264812912259783, 0.09875170283374332, -0...","[-0.03339471357564131, -0.06570093830426534, -...",[unknown],[unknown]
2,31376,27506397,7673df8696818aae,909885948267253760,en,0,1,0,0,This week is slowing turning into a total slim...,...,None,0,NaN,week slowing turning total slimming world write,"[This, week, is, slowing, turning, into, a, to...","[week, slowing, turning, total, slimming, worl...","[-0.07041776660256661, 0.08685002934474212, -0...","[-0.040115516632795334, -0.16007946591292108, ...",[unknown],[unknown]
3,31377,14411725,47517fbbeca71972,574217292415893505,en,1,0,0,5,It is poor people people with disabilities and...,...,None,0,NaN,poor people people disabilities women beared b...,"[It, is, poor, people, people, with, disabilit...","[poor, people, people, disabilities, women, be...","[0.12500025871850085, 0.09377818144275807, -0....","[-0.06087548720339934, -0.03791896156811466, -...",[unknown],[unknown]
4,31377,43148115,791e00bcadc4615f,1331698503379267584,en,1,0,0,0,"@RobertCorp Steel yourself, it’s a cracker",...,None,0,NaN,robertcorp steel cracker,"[@, RobertCorp, Steel, yourself, ,, it, ’, s, ...","[robertcorp, steel, cracker]","[-0.14062001772690563, 0.1959332786500454, -0....","[0.10511746505896251, 0.10234808921813965, 0.0...",[unknown],[unknown]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267784,31363,1072767550726000640,0af014accd6f6e99,1331893227201425409,en,0,0,1,0,@forestsandwood @RishiSunak @StuGood3 Is that ...,...,None,0,1.0,forestsandwood rishisunak stugood3 baby sitka ...,"[@, forestsandwood, @, RishiSunak, @, StuGood3...","[forestsandwood, rishisunak, stugood3, baby, s...","[-0.01569323461695478, 0.07200790200972576, -0...","[-0.06944699078027544, 0.08604351578858392, 0....",[person],"[rishi, sunak]"
1267785,38856,781287781,503666777ae86494,1581964504304230400,en,0,0,0,0,@AdamBienkov @dontbrexitfixit Where is she the...,...,None,0,NaN,adambienkov dontbrexitfixit quivering wreck pm...,"[@, AdamBienkov, @, dontbrexitfixit, Where, is...","[adambienkov, dontbrexitfixit, quivering, wrec...","[-0.03016398249421903, 0.07999796035917515, -0...","[-0.043783893420671426, 0.016469326698117785, ...",[unknown],[unknown]
1267786,38856,899693078,381b5744987ae0bd,897829201847439360,en,1,0,0,0,Our latest #Bath Business email is out!\n\nRea...,...,None,0,NaN,latest bath business email outread httpstcornb...,"[Our, latest, #, Bath, Business, email, is, ou...","[latest, bath, business, email, outread, https...","[0.07085830328986048, 0.12979740157723427, -0....","[-0.16043231139580408, -0.08681366670255859, -...",[unknown],[unknown]
1267787,38857,28104649,70392b0b6ad1f95b,562389444188450817,en,0,0,0,0,"Been sat here since 7.30 (pm, I do have a job)...",...,None,0,NaN,sat pm job total spent sky changed zahas perma...,"[Been, sat, here, since, 7.30, (, pm, ,, I, do...","[sat, pm, job, total, spent, sky, changed, zah...","[0.05067618466273416, 0.0406289599777665, -0.2...","[-0.15093560107052326, -0.16009908608393744, 0..."

In [40]:
#convert context field from string to json
def stringtojson(row):
    string = row['context']
    return json.loads(string)

In [42]:
#convert empty context values to []
for index, item in enumerate(data['context']):
    if item is None:
        data.loc[index, 'context'] = '[]'

KeyboardInterrupt: 

In [ ]:
data['context']=data.apply(stringtojson,axis=1)

#extract the name from the first pair. if null replace with 'unkonwn'
data['context_domain_name'] = [entry[0]['domain']['name'] if len(entry)>1 else 'unknown' for entry in data['context']]

#lowercasing
data['context_domain_name']=[entry.lower() for entry in data['context_domain_name']]

#tokenisation
data['context_domain_name']=[word_tokenize(entry) for entry in data['context_domain_name']]

In [22]:
data

,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,tweet_original,...,rowid,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_original,tweet_tokenized_preprocessed,tweet_word2vec_original,tweet_word2vec_preprocessed,context_domain_name
0,31371,58555376,1a77876636535465,574226197921595394,en,0,0,0,1,Pen 26 Garn 3\n\nBp in the bag\n\n#pitmen\n\n#fb,...,156267,None,0,NaN,pen garn bp bagpitmenfb,"[Pen, 26, Garn, 3, Bp, in, the, bag, #, pitmen...","[pen, garn, bp, bagpitmenfb]","[0.09151028875600208, 0.04513007774949074, -0....","[0.004402711604294988, -0.012395232915878296, ...",[unknown]
1,31376,25727727,1da00c8852cc9da2,574217464243949568,en,2,0,1,0,Definitely going to look like a total wally on...,...,156293,None,0,NaN,definitely going look like total wally train c...,"[Definitely, going, to, look, like, a, total, ...","[definitely, going, look, like, total, wally, ...","[-0.03264812912259783, 0.09875170283374332, -0...","[-0.03339471357564131, -0.06570093830426534, -...",[unknown]
2,31376,27506397,7673df8696818aae,909885948267253760,en,0,1,0,0,This week is slowing turning into a total slim...,...,156294,None,0,NaN,week slowing turning total slimming world write,"[This, week, is, slowing, turning, into, a, to...","[week, slowing, turning, total, slimming, worl...","[-0.07041776660256661, 0.08685002934474212, -0...","[-0.040115516632795334, -0.16007946591292108, ...",[unknown]
3,31377,14411725,47517fbbeca71972,574217292415893505,en,1,0,0,5,It is poor people people with disabilities and...,...,156297,None,0,NaN,poor people people disabilities women beared b...,"[It, is, poor, people, people, with, disabilit...","[poor, people, people, disabilities, women, be...","[0.12500025871850085, 0.09377818144275807, -0....","[-0.06087548720339934, -0.03791896156811466, -...",[unknown]
4,31377,43148115,791e00bcadc4615f,1331698503379267584,en,1,0,0,0,"@RobertCorp Steel yourself, it’s a cracker",...,156298,None,0,NaN,robertcorp steel cracker,"[@, RobertCorp, Steel, yourself, ,, it, ’, s, ...","[robertcorp, steel, cracker]","[-0.14062001772690563, 0.1959332786500454, -0....","[0.10511746505896251, 0.10234808921813965, 0.0...",[unknown]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267784,31363,1072767550726000640,0af014accd6f6e99,1331893227201425409,en,0,0,1,0,@forestsandwood @RishiSunak @StuGood3 Is that ...,...,156230,None,0,1.0,forestsandwood rishisunak stugood3 baby sitka ...,"[@, forestsandwood, @, RishiSunak, @, StuGood3...","[forestsandwood, rishisunak, stugood3, baby, s...","[-0.01569323461695478, 0.07200790200972576, -0...","[-0.06944699078027544, 0.08604351578858392, 0....",[person]
1267785,38856,781287781,503666777ae86494,1581964504304230400,en,0,0,0,0,@AdamBienkov @dontbrexitfixit Where is she the...,...,193634,None,0,NaN,adambienkov dontbrexitfixit quivering wreck pm...,"[@, AdamBienkov, @, dontbrexitfixit, Where, is...","[adambienkov, dontbrexitfixit, quivering, wrec...","[-0.03016398249421903, 0.07999796035917515, -0...","[-0.043783893420671426, 0.016469326698117785, ...",[unknown]
1267786,38856,899693078,381b5744987ae0bd,897829201847439360,en,1,0,0,0,Our latest #Bath Business email is out!\n\nRea...,...,193635,None,0,NaN,latest bath business email outread httpstcornb...,"[Our, latest, #, Bath, Business, email, is, ou...","[latest, bath, business, email, outread, https...","[0.07085830328986048, 0.12979740157723427, -0....","[-0.16043231139580408, -0.08681366670255859, -...",[unknown]
1267787,38857,28104649,70392b0b6ad1f95b,562389444188450817,en,0,0,0,0,"Been sat here since 7.30 (pm, I do have a job)...",...,193636,None,0,NaN,sat pm job total spent sky changed zahas perma...,"[Been, sat, here, since, 7.30, (, pm, ,, I, do...","[sat, pm, job, total, spent, sky, changed, zah...","[0.05067618466273416, 0.0406289599777665, -0.2...","[-0.15093560107052326, -0.16009908608393744, 0...",[unknown]


In [23]:
data['context_domain_name'].value_counts()

context_domain_name
[unknown]                               998181
[person]                                 69757
[tv, shows]                              50580
[entities, [, entity, service, ]]        24177
[brand, vertical]                        23691
[interests, and, hobbies, vertical]      17370
[unified, twitter, taxonomy]             15681
[ongoing, news, story]                   11004
[sports, event]                          10730
[events, [, entity, service, ]]           8518
[place]                                   5282
[brand]                                   4804
[business, taxonomy]                      4390
[political, body]                         3964
[sports, team]                            3862
[interests, and, hobbies, category]       3231
[sport]                                   3211
[news, vertical]                          1563
[interests, and, hobbies]                 1373
[multimedia, franchise]                   1242
[movie]                                 

In [25]:
#filter out manually labelled data for creating train and test set
train_test_data = data.dropna(subset=['related_manual'])

In [26]:
train_test_data

,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,tweet_original,...,rowid,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_original,tweet_tokenized_preprocessed,tweet_word2vec_original,tweet_word2vec_preprocessed,context_domain_name
55,31449,206116791,28679b23ed15b380,1331001975123619842,en,0,0,0,1,Housing Executive estimates cost of becoming c...,...,156659,None,1,1.0,housing executive estimates cost carbon neutra...,"[Housing, Executive, estimates, cost, of, beco...","[housing, executive, estimates, cost, carbon, ...","[-0.016047484486510877, 0.0352895640228924, -0...","[-0.15930135629605502, 0.09321914659813046, -0...",[unknown]
66,31495,1234686228,3bc1b6cfd27ef7f6,1330649340692533257,en,0,0,1,0,@TaeSavesLives @WoollyMaggot What about biodiv...,...,156888,None,1,1.0,taesaveslives woollymaggot biodiversity carbon...,"[@, TaeSavesLives, @, WoollyMaggot, What, abou...","[taesaveslives, woollymaggot, biodiversity, ca...","[0.03088920424731138, 0.10161545368221898, -0....","[-0.25625161315772194, 0.10566816953095523, 0....",[unknown]
79,31511,811682045294419969,315b740b108481f6,1330581930434322436,en,4,0,2,1,Bury Council - Climate Emergency - carbon neut...,...,156970,None,1,1.0,bury council climate emergency carbon neutral ...,"[Bury, Council, -, Climate, Emergency, -, carb...","[bury, council, climate, emergency, carbon, ne...","[0.005070036235782835, -0.005466167608069049, ...","[-0.26287255224955375, 0.09272507118741767, -0...","[entities, [, entity, service, ]]"
102,31553,995666737,15d90627b230db4d,1330181990830780420,en,0,0,0,0,Another despicable act by @HS2ltd The best car...,...,157180,None,1,1.0,despicable act hs2ltd best carbon capture matu...,"[Another, despicable, act, by, @, HS2ltd, The,...","[despicable, act, hs2ltd, best, carbon, captur...","[0.025153253152633885, 0.03523394518664905, -0...","[-0.08783161461663742, 0.052445119324450694, 0...","[entities, [, entity, service, ]]"
120,31571,2174311068,42d0cf7d49d27c95,1330097451559227392,en,0,0,0,0,@fayrebumpplayon @Karmageddon67 @BrettDwyer1 @...,...,157269,None,1,1.0,fayrebumpplayon karmageddon67 brettdwyer1 exti...,"[@, fayrebumpplayon, @, Karmageddon67, @, Bret...","[fayrebumpplayon, karmageddon67, brettdwyer1, ...","[-0.015938892478256354, 0.10402628732845187, -...","[-0.11855451817708937, 0.10559250033375892, -0...",[unknown]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267705,31128,35538459,7f55e839e1715da2,1334406016700735489,en,2,0,0,1,The UK's new National Investment Bank (NIB) mu...,...,155054,None,1,1.0,uks new national investment bank nib strong ne...,"[The, UK, 's, new, National, Investment, Bank,...","[uks, new, national, investment, bank, nib, st...","[-0.020250934897465447, 0.0715447610823949, -0...","[-0.22006224028766155, -0.005232255794107914, ...","[entities, [, entity, service, ]]"
1267719,31154,896405876,25d3e991f5637f5a,1334065438771261441,en,7,0,0,2,Please retweet this fantastic project @Floodpl...,...,155187,None,1,1.0,retweet fantastic project floodplainmead ferti...,"[Please, retweet, this, fantastic, project, @,...","[retweet, fantastic, project, floodplainmead, ...","[0.08001487965408999, 0.07588679747035106, -0....","[-0.037412236063656484, 0.05708902481604706, 0...",[unknown]
1267725,31181,545269148,3a8a2c667faaf9ba,1333753783030001666,en,0,0,0,0,Manufacturing steel is a carbon intensive proc...,...,155321,None,0,1.0,manufacturing steel carbon intensive process t...,"[Manufacturing, steel, is, a, carbon, intensiv...","[manufacturing, steel, carbon, intensive, proc...","[-0.02302748753643799, 0.09524179493054384, -0...","[-0.06356183784082532, 0.07910259268246592, -0...",[unknown]
1267751,31269,8155242,07e9c7d1954fff64,1332740744243781634,en,3,1,10,0,love a challenge so thought I’d incorporate al...,...,155757,None,0,0.0,love challenge thought id incorporate cycle ro...,"[love, a, challenge, so, thought, I, ’, d, inc..

In [27]:
#save unlabelled data to a separate dataset
remaining_data = data[~data.index.isin(train_test_data.index)]

In [28]:
train_test_data['related_manual'].value_counts()

related_manual
1.0    1250
0.0    1250
Name: count, dtype: int64

In [29]:
data_columns = train_test_data.drop(columns=['related_manual'])
target_column = train_test_data['related_manual']

In [30]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(data_columns,target_column, test_size=0.2, stratify=train_test_data['related_manual'])

In [32]:
train_df = pd.concat([x_train, y_train], axis=1)

In [33]:
train_df['related_manual'].value_counts()

related_manual
0.0    1000
1.0    1000
Name: count, dtype: int64

In [34]:
test_df = pd.concat([x_test, y_test], axis=1)

In [35]:
test_df['related_manual'].value_counts()

related_manual
1.0    250
0.0    250
Name: count, dtype: int64

In [42]:
train_df = train_df.reset_index()

In [43]:
test_df = test_df.reset_index()

In [6]:
remaining_data = remaining_data.reset_index()

In [71]:
%%time
#for train data, generate word2vec embedding for context
word2vector_model_train = gensim.models.Word2Vec(
            train_df['context_domain_name'],
            size=300,
            window=1,
            min_count=2,                                  
            sg = 1,
            hs = 0,
            negative = 10,
            workers= 32,
            seed = 34
) 
word2vector_model_train.train(train_df['context_domain_name'], total_examples= len(train_df['context_domain_name']), epochs=20)

CPU times: user 187 ms, sys: 66.3 ms, total: 253 ms
Wall time: 212 ms


(8568, 58260)

In [72]:
%%time
context_vectors_train = np.zeros((len(train_df['context_domain_name']), 300)) 
for i in train_df.index:
    context_vectors_train[i,:] = get_word_vector(train_df['context_domain_name'][i], 300, word2vector_model_train)

CPU times: user 55.4 ms, sys: 4.96 ms, total: 60.3 ms
Wall time: 59.5 ms


/var/folders/yp/w3wbm1755g3dkb_6mbfzm41r0000gn/T/ipykernel_34846/4174304635.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += model[word].reshape((1, size))


In [73]:
arr = {"lists": list(context_vectors_train)}
context_vectors_train = pd.DataFrame(arr, columns=['lists'])
train_df['context_word2vec'] = context_vectors_train['lists']
train_df

,level_0,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,...,ccs_score,rowid,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_preprocessed,tweet_word2vec_preprocessed,context_domain_name,context_word2vec
0,0,152,948621816830005248,62a2a7f86cd9a5b4,1641001927528808450,en,15,0,1,0,...,0,798,None,0,1.0,nonelectricfuel mowers2 dont mow let wild lown...,"[nonelectricfuel, mowers2, dont, mow, let, wil...","[-0.16273461307088535, -0.044132084446027876, ...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
1,15,746,2436060810,72e2e7dabe2927ae,1634147123087245314,en,4,0,2,0,...,0,3754,None,0,1.0,youll need explain carbon sink comment commerc...,"[youll, need, explain, carbon, sink, comment, ...","[-0.2758036535233259, -0.023045434791129082, 0...","[interests, and, hobbies, category]","[0.04123735520988703, 0.09485122375190258, 0.0..."
2,18,755,1444658250218196998,17aacd85e89124d8,1634124524227575811,en,0,0,1,0,...,0,3800,None,0,1.0,excellent clip simon yes difficult value maybe...,"[excellent, clip, simon, yes, difficult, value...","[-0.2053347546219205, -0.0904396822831283, -0....",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
3,19,983,1185731611,513372d74ee7530b,1631179311767068674,en,10,0,1,4,...,0,4934,None,0,1.0,looking different habitats provide stack biodi...,"[looking, different, habitats, provide, stack,...","[-0.12481054086238146, -0.09381166743114591, -...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
4,24,1008,1029270222606352390,4efd6fc8c50fae33,1630833854809350144,en,3,0,0,0,...,0,5058,None,0,1.0,carbon sequestration calcs opportunities orgs ...,"[carbon, sequestration, calcs, opportunities, ...","[-0.18592729129725033, -0.027451477233423956, ...","[entities, [, entity, service, ]]","[0.03573186472058296, 0.0645165927708149, 0.07..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,68983,394137,54351001,5f6514b0d7b1bc4c,987339322516008960,en,3,1,0,2,...,1,1170303,None,1,0.0,ge manifesto prioritise brownfield sites prote...,"[ge, manifesto, prioritise, brownfield, sites,...","[-0.16799876790331758, -0.2312537778330886, -0...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
1996,69364,400137,2248691238,27d84a92c9d52794,980137961122648064,en,0,0,1,0,...,1,1176352,None,0,1.0,responsible suppliers maintaining forestland f...,"[responsible, suppliers, maintaining, forestla...","[-0.23160432456504731, -0.04003317663002582, -...",[person],"[-0.0011677509173750877, -0.001496690209023654..."
1997,70354,417666,84869145,2ee88a47c191b2d5,960647942913888256,en,0,0,0,0,...,1,1194020,None,1,0.0,minister forestry environment siti nuraya baka...,"[minister, forestry, environment, siti, nuraya...","[-0.29602330757511985, 0.06986505383004744, -0...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
1998,70573,421436,52693836,07e9c7d1954fff64,956445654682882048,en,15,5,11,13,...,1,1197813,None,1,0.0,good luck opposing today terrible foothold dis...,"[good, luck, opposing, today, terrible, footho...","[-0.11234844531863927, -0.005403078719973564, ...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."


In [74]:
%%time
#for test data, generate word2vec embedding for context
word2vector_model_test = gensim.models.Word2Vec(
            test_df['context_domain_name'],
            size=300,
            window=1,
            min_count=2,                                  
            sg = 1,
            hs = 0,
            negative = 10,
            workers= 32,
            seed = 34
) 
word2vector_model_test.train(test_df['context_domain_name'], total_examples= len(test_df['context_domain_name']), epochs=20)

CPU times: user 127 ms, sys: 52.9 ms, total: 180 ms
Wall time: 147 ms


(2072, 14600)

In [75]:
%%time
context_vectors_test = np.zeros((len(test_df['context_domain_name']), 300)) 
for i in test_df.index:
    context_vectors_test[i,:] = get_word_vector(test_df['context_domain_name'][i], 300, word2vector_model_test)

CPU times: user 17.8 ms, sys: 5.81 ms, total: 23.6 ms
Wall time: 18.6 ms


/var/folders/yp/w3wbm1755g3dkb_6mbfzm41r0000gn/T/ipykernel_34846/4174304635.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += model[word].reshape((1, size))


In [76]:
arr = {"lists": list(context_vectors_test)}
context_vectors_test = pd.DataFrame(arr, columns=['lists'])
test_df['context_word2vec'] = context_vectors_test['lists']
test_df

,level_0,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,...,ccs_score,rowid,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_preprocessed,tweet_word2vec_preprocessed,context_domain_name,context_word2vec
0,1,196,1150771325721546754,7d7bdec12d2549d4,1640409818140639232,en,6,0,0,1,...,0,1019,None,0,1.0,feb ground broken new bluecoat academy carring...,"[feb, ground, broken, new, bluecoat, academy, ...","[-0.17708985341295344, -0.09123214083755839, 0...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
1,20,1000,1233114093629976582,25d3e991f5637f5a,1630937495616495617,en,1,0,0,0,...,0,5018,None,0,1.0,seagrass vital habitat cuttlefish species func...,"[seagrass, vital, habitat, cuttlefish, species...","[-0.32027924060821533, -0.1663591515272856, 0....",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
2,37,2340,20662440,25d3e991f5637f5a,1614419209798258688,en,17,0,1,2,...,0,11679,None,0,1.0,know buying land improve biodiversity carbon s...,"[know, buying, land, improve, biodiversity, ca...","[-0.24470218703808153, -0.021547190833028734, ...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
3,44,3027,265276284,7ae9e2f2ff7a87cd,1605479146288664576,en,1,0,0,0,...,0,15099,None,0,1.0,aye theres rub tradeoffs optimising annual car...,"[aye, theres, rub, tradeoffs, optimising, annu...","[-0.2432087640399518, -0.07931409381410998, -0...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
4,126,300,20662440,06168d1feda43857,1639341574445309952,en,0,0,1,0,...,20,1535,None,1,1.0,ccs knowledge sticking carbon capture fossil f...,"[ccs, knowledge, sticking, carbon, capture, fo...","[-0.240239797733151, -0.028277638601139188, -0...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,68908,392963,187602448,53b67b1d1cc81a51,988687454143205377,en,2,0,0,0,...,1,1169123,None,1,0.0,absolutely silos getting deepertotal new envir...,"[absolutely, silos, getting, deepertotal, new,...","[-0.12502856420663497, 0.07465614553075284, -0...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
496,70708,423866,388793955,25d3e991f5637f5a,954077360709619712,en,1,0,0,0,...,1,1200261,None,1,0.0,total waste time year olds regimented learning...,"[total, waste, time, year, olds, regimented, l...","[-0.19740079971961677, -0.09446627937722951, -...",[unknown],"[0.00023176193644758314, -0.001168469781987369..."
497,71356,194500,2831955446,720081c4c09a4a84,1198915726486228992,en,1,0,0,0,...,1,870955,None,1,0.0,bp needs resolve environmental issues goes phi...,"[bp, needs, resolve, environmental, issues, go...","[-0.14111729338765144, -0.16012865105377777, 0...","[brand, vertical]","[-0.007221590261906385, 0.0060702532064169645,..."
498,71358,194851,2273675919,1f36c2b60fbc98ac,1198533532781568001,en,65,1,5,3,...,1,872356,None,1,0.0,watched blues leicester game defensive shift c...,"[watched, blues, leicester, game, defensive, s...","[-0.032119659171439705, 0.09410803392529488, -...",[brand],"[-0.00477095041424036, 0.004586540162563324, 0..."


In [4]:
%%time
#for unlabelled data, generate word2vec embedding for context
word2vector_model_domain_1 = gensim.models.Word2Vec(
            remaining_data['context_domain_name'],
            size=300,
            window=1,
            min_count=2,                                  
            sg = 1,
            hs = 0,
            negative = 10,
            workers= 32,
            seed = 34
) 
word2vector_model_domain_1.train(remaining_data['context_domain_name'], total_examples= len(remaining_data['context_domain_name']), epochs=20)

CPU times: user 30.4 s, sys: 813 ms, total: 31.2 s
Wall time: 28.2 s


(5069397, 32343740)

In [10]:
%%time
context_vectors_remaining = np.zeros((len(remaining_data['context_domain_name']), 300)) 
for i in remaining_data.index:
    context_vectors_remaining[i,:] = get_word_vector(remaining_data['context_domain_name'][i], 300, word2vector_model_domain_1)

/var/folders/yp/w3wbm1755g3dkb_6mbfzm41r0000gn/T/ipykernel_1100/4174304635.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector += model[word].reshape((1, size))


CPU times: user 24 s, sys: 1.24 s, total: 25.3 s
Wall time: 25.3 s


In [11]:
arr = {"lists": list(context_vectors_remaining)}
context_vectors_remaining = pd.DataFrame(arr, columns=['lists'])
remaining_data['context_word2vec'] = context_vectors_remaining['lists']
remaining_data

,level_0,index,author_id,geo,tweetid,lang,like_count,quote_count,reply_count,retweet_count,...,relevant_manualscore,related_score_based,related_manual,tweet_preprocessed,tweet_tokenized_original,tweet_tokenized_preprocessed,tweet_word2vec_original,tweet_word2vec_preprocessed,context_domain_name,context_word2vec
0,0,31371,58555376,1a77876636535465,574226197921595394,en,0,0,0,1,...,None,0,NaN,pen garn bp bagpitmenfb,"[Pen, 26, Garn, 3, Bp, in, the, bag, #, pitmen...","[pen, garn, bp, bagpitmenfb]","[0.09151028875600208, 0.04513007774949074, -0....","[0.004402711604294988, -0.012395232915878296, ...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
1,1,31376,25727727,1da00c8852cc9da2,574217464243949568,en,2,0,1,0,...,None,0,NaN,definitely going look like total wally train c...,"[Definitely, going, to, look, like, a, total, ...","[definitely, going, look, like, total, wally, ...","[-0.03264812912259783, 0.09875170283374332, -0...","[-0.03339471357564131, -0.06570093830426534, -...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
2,2,31376,27506397,7673df8696818aae,909885948267253760,en,0,1,0,0,...,None,0,NaN,week slowing turning total slimming world write,"[This, week, is, slowing, turning, into, a, to...","[week, slowing, turning, total, slimming, worl...","[-0.07041776660256661, 0.08685002934474212, -0...","[-0.040115516632795334, -0.16007946591292108, ...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
3,3,31377,14411725,47517fbbeca71972,574217292415893505,en,1,0,0,5,...,None,0,NaN,poor people people disabilities women beared b...,"[It, is, poor, people, people, with, disabilit...","[poor, people, people, disabilities, women, be...","[0.12500025871850085, 0.09377818144275807, -0....","[-0.06087548720339934, -0.03791896156811466, -...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
4,4,31377,43148115,791e00bcadc4615f,1331698503379267584,en,1,0,0,0,...,None,0,NaN,robertcorp steel cracker,"[@, RobertCorp, Steel, yourself, ,, it, ’, s, ...","[robertcorp, steel, cracker]","[-0.14062001772690563, 0.1959332786500454, -0....","[0.10511746505896251, 0.10234808921813965, 0.0...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265284,1267783,31360,948621816830005248,0af014accd6f6e99,1588456319157432321,qam,0,0,0,0,...,None,0,NaN,brcalvert kavitabp allwecansave,"[@, brcalvert, @, kavita_bp, @, allwecansave]","[brcalvert, kavitabp, allwecansave]","[-0.33525748923420906, 0.23291491344571114, -0...","[0.032170288264751434, -0.2934260964393616, 0....",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
1265285,1267785,38856,781287781,503666777ae86494,1581964504304230400,en,0,0,0,0,...,None,0,NaN,adambienkov dontbrexitfixit quivering wreck pm...,"[@, AdamBienkov, @, dontbrexitfixit, Where, is...","[adambienkov, dontbrexitfixit, quivering, wrec...","[-0.03016398249421903, 0.07999796035917515, -0...","[-0.043783893420671426, 0.016469326698117785, ...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
1265286,1267786,38856,899693078,381b5744987ae0bd,897829201847439360,en,1,0,0,0,...,None,0,NaN,latest bath business email outread httpstcornb...,"[Our, latest, #, Bath, Business, email, is, ou...","[latest, bath, business, email, outread, https...","[0.07085830328986048, 0.12979740157723427, -0....","[-0.16043231139580408, -0.08681366670255859, -...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."
1265287,1267787,38857,28104649,70392b0b6ad1f95b,562389444188450817,en,0,0,0,0,...,None,0,NaN,sat pm job total spent sky changed zahas perma...,"[Been, sat, here, since, 7.30, (, pm, ,, I, do...","[sat, pm, job, total, spent, sky, changed, zah...","[0.05067618466273416, 0.0406289599777665, -0.2...","[-0.15093560107052326, -0.16009908608393744, 0...",[unknown],"[-0.00016023182251956314, 4.825825817533769e-0..."


In [77]:
train_df_cleaned = train_df.drop(columns=['tweet_original'])
test_df_cleaned = test_df.drop(columns=['tweet_original'])

In [ ]:
#save raw tweet (labelled) dataset
train_df.to_pickle('./train_df.pkl')
test_df.to_pickle('./test_df.pkl')

In [ ]:
#save clean tweet (labelled) dataset
train_df_cleaned.to_pickle('./train_df_cleaned.pkl')
test_df_cleaned.to_pickle('./test_df_cleaned.pkl')

In [12]:
#save unlabelled dataset
remaining_data.to_pickle('./remaining_df.pkl')